BenchmarkTools.jl (@btime) or Chairmarks.jl (@b)

In [1]:
from juliacall import Main as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
%load_ext juliacall

In [4]:
%%julia

import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.5.0
    Updating `~/anaconda3/envs/julia_hep_2024/julia_env/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
    Updating `~/anaconda3/envs/julia_hep_2024/julia_env/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [9abbd945] + Profile
Precompiling project...
  ✓ BenchmarkTools
  1 dependency successfully precompiled in 4 seconds. 123 already precompiled.


In [8]:
%%julia

using Pkg
Pkg.add("UnROOT")
using UnROOT
using AwkwardArray

   Resolving package versions...
  No Changes to `~/anaconda3/envs/julia_hep_2024/julia_env/Project.toml`
  No Changes to `~/anaconda3/envs/julia_hep_2024/julia_env/Manifest.toml`


In [9]:
%%julia

tree = ROOTFile("./data/SMHiggsToZZTo4L.root")
events = LazyTree(tree, "Events")

 Row │ Electron_mass    nElectron  luminosityBlock  nMuon   Electron_phi     M ⋯
     │ SubArray{Float3  UInt32     UInt32           UInt32  SubArray{Float3  F ⋯
─────┼──────────────────────────────────────────────────────────────────────────
 1   │ []               0          156              3       []               - ⋯
 2   │ [0.00544,        4          156              0       [0.134, -1       2 ⋯
 3   │ [-0.00609,       2          156              0       [2.18, 1.6       - ⋯
 4   │ [-0.00123]       1          156              7       [-0.643]         - ⋯
 5   │ [0.0117, 0       4          156              0       [1.01, -1.       1 ⋯
 6   │ [-0.00183]       1          156              2       [-0.497]         - ⋯
 7   │ [-0.00183]       1          156              1       [-1.47]          - ⋯
 8   │ [-0.00216]       1          156              0       [-0.633]         - ⋯
 9   │ [-0.0128,        4          156              0       [2.79, -2.       - ⋯
 10  │ []               0   

In [6]:
%%julia

using Base.Threads
println(Threads.nthreads())

4


In [11]:
%%julia

using LorentzVectorHEP
using AwkwardArray
using Base.Threads

function main_looper(events)
    array = AwkwardArray.PrimitiveArray{Float64}()
    lock_obj = ReentrantLock()  # Create a lock object to control access to shared array

    @threads for i in 1:length(events)
        evt = events[i]

        # Destructure the necessary fields from the event
        (; Muon_charge, Muon_pt, Muon_eta, Muon_phi, Muon_mass) = evt

        # Skip event if it doesn't meet the required conditions
        if length(Muon_charge) != 4 || sum(Muon_charge) != 0
            continue
        end

        # Create Lorentz vectors for the muons and calculate the Higgs mass
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)

        # Use lock to safely push! into the shared array
        lock(lock_obj)  # Explicitly lock before modifying shared data
        try
            push!(array, higgs_mass)
        finally
            unlock(lock_obj)  # Ensure the lock is always released
        end
    end

    return array
end

main_looper (generic function with 1 method)

In [12]:
%%julia

array = @time main_looper(events)

  0.294637 seconds (222.10 k allocations: 101.787 MiB, 5.02% gc time, 234.50% compilation time)


20525-element AwkwardArray.PrimitiveArray{Float64, Vector{Float64}, :default}:
 125.12303161621094
 123.90653991699219
 124.15757751464844
 122.6549301147461
 125.26071166992188
 124.77593994140625
 124.20553588867188
 124.42249298095703
 110.03680419921875
 124.46846008300781
   ⋮
 127.44519805908203
 125.3969497680664
 124.91043090820312
 124.92266845703125
 103.70930480957031
 124.39176940917969
 125.422119140625
 125.07860565185547
 123.24031829833984

In [13]:
%%julia

array = @btime main_looper(events)

  36.089 ms (22960 allocations: 59.59 MiB)


20525-element AwkwardArray.PrimitiveArray{Float64, Vector{Float64}, :default}:
 125.30791473388672
 122.7776107788086
 113.92278289794922
 124.49745178222656
 115.7300033569336
 124.36397552490234
  77.15567779541016
 123.48483276367188
  50.24703598022461
 124.37024688720703
   ⋮
 127.44519805908203
 125.3969497680664
 124.91043090820312
 124.92266845703125
 103.70930480957031
 124.39176940917969
 125.422119140625
 125.07860565185547
 123.24031829833984

In [15]:
%%julia

import Pkg; Pkg.add("Chairmarks")
using Chairmarks

   Resolving package versions...
   Installed Chairmarks ─ v1.2.2
    Updating `~/anaconda3/envs/julia_hep_2024/julia_env/Project.toml`
  [0ca39b1e] + Chairmarks v1.2.2
    Updating `~/anaconda3/envs/julia_hep_2024/julia_env/Manifest.toml`
  [0ca39b1e] + Chairmarks v1.2.2
Precompiling project...
  ✓ Chairmarks
  ✓ Chairmarks → StatisticsChairmarksExt
  2 dependencies successfully precompiled in 5 seconds. 124 already precompiled.


In [16]:
%%julia

array = @b main_looper(events)

88.257 ms (23995 allocs: 87.739 MiB, 34.15% gc time)

In [17]:
%%julia

array = @b main_looper(events)

47.156 ms (23995 allocs: 87.739 MiB)